In [1]:
import pandas as pd
import numpy as np

# import necessary file
from datetime import datetime
# from tqdm.notebook import tqdm
from tqdm import tqdm
tqdm.pandas(desc="Loading...")
import swifter

# Setting the pyecharts config
from pyecharts.globals import CurrentConfig, NotebookType
CurrentConfig.NOTEBOOK_TYPE = NotebookType.JUPYTER_LAB

# setting pyecharts' charts
from pyecharts.charts import Funnel, Bar, Line, Grid
from pyecharts import options as opts
from pyecharts.globals import ThemeType

In [3]:
#importing all data
df = pd.read_csv('Data/UserBehavior.csv', header=None, parse_dates=True)
print('Loading Finished! The file length is {} rows'.format(len(df)))

Loading Finished! The file length is 100150807 rows


In [4]:
# 重新命名欄位
print('Renaming Columns...')
df.columns = ['User_Id', 'Item_Id', 'Category_Id', 'Behavior', 'Time']
print('Renamed column Finished...')

Renaming Columns...
Renamed column Finished...


In [5]:
df_behav = df.loc[:, ['User_Id', 'Behavior', 'Time']]

In [7]:
df_behav['Behavior'] = df_behav.Behavior.astype('category')
df_behav['User_Id'] = df_behav.User_Id.astype('category')

In [12]:
# 清洗超出時間範圍的數據
print('Filtering data...')
df_behav['Time'] = pd.to_datetime(df_behav['Time'], unit="s")
mask_1 = df_behav['Time'] >= datetime(2017,11,25)
mask_2 = df_behav['Time'] < datetime(2017,12,3)
df_behav = df_behav[mask_1 & mask_2]
print('Filter data Finished...')

Filtering data...
Filter data Finished...


In [20]:
# df_behav['Date'] = df_behav.Time.progress_apply(lambda t: t.dt.strftime("%Y-%m-%d").astype('category'))
# df_behav['Hour'] = df_behav.Time.dt.strftime("%H").astype('category')
# df_behav['Week'] = df_behav.Time.dt.strftime("%A").astype('category')

In [21]:
df_behav.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 86953525 entries, 2 to 100150794
Data columns (total 3 columns):
 #   Column    Dtype         
---  ------    -----         
 0   User_Id   category      
 1   Behavior  category      
 2   Time      datetime64[ns]
dtypes: category(2), datetime64[ns](1)
memory usage: 1.7 GB


In [41]:
%time df_behav.head()

CPU times: user 715 µs, sys: 68 µs, total: 783 µs
Wall time: 746 µs


,User_Id,Behavior,Time
2,1,pv,2017-11-25 01:21:25
3,1,pv,2017-11-25 07:04:53
4,1,pv,2017-11-25 07:49:06
5,1,pv,2017-11-25 13:28:01
6,1,pv,2017-11-25 21:22:22


In [42]:
%time df_week = df_behav[(df_behav['Time'] >= pd.to_datetime("2017-11-26")) & (df_behav['Time'] < pd.to_datetime("2017-12-03"))]
df_week['Date'] = df_week.Time.dt.strftime('%Y-%m-%d').astype('category')
df_week = df_week.drop('Time')

CPU times: user 3.02 s, sys: 1.51 s, total: 4.53 s
Wall time: 4.89 s


KeyboardInterrupt: 

In [37]:
def week_behav():
    '''
    先把日期過濾成一週，再把數據進行透視分析以取出各項數據。
    '''
    df_week = df_behav[(df_behav['Time'] >= pd.to_datetime("2017-11-26")) & (df_behav['Time'] < pd.to_datetime("2017-12-03"))]
    df_week['Date'] = df_week.Time.dt.strftime('%Y-%m-%d')
    df_week = df_week.drop('Time')
    behavior = df_week.pivot_table(index="Date", columns="Behavior", aggfunc=['count']).iloc[:, :4]
    behavior.columns = ['buy', 'cart', 'fav', 'pv']
    behavior = behavior.reset_index()[col].to_list()
    return df_week
w_behav = week_behav()

KeyboardInterrupt: 

In [25]:
def week_chart():
    week_day = ['Sun', 'Mon', 'Tues', 'Wed', 'Thur', 'Fri', 'Sat']
    bar = (
        Bar()
        .add_xaxis(week_day)
        .add_yaxis("購買量", week_behav("buy"), stack="stack_1", category_gap=60)
        .add_yaxis("購物車數量", week_behav("cart"), stack="stack_1", category_gap=60)
        .add_yaxis("我的最愛", week_behav("fav"), stack="stack_1", category_gap=60)
        .set_series_opts(
            label_opts=opts.LabelOpts(is_show=True)
        )
        .set_global_opts(
            title_opts=opts.TitleOpts(
                title="用戶變化週期曲線"
            ),
            legend_opts=opts.LegendOpts(
                pos_top="5%"
            )
        )
    )
    
    line = (
        Line()
        .add_xaxis(week_day)
        .add_yaxis("瀏覽人次", week_behav("pv"), yaxis_index=1)
        .set_global_opts(
            title_opts=opts.TitleOpts(
                title="用戶瀏覽人次",
                pos_top="48%",
            ),
            legend_opts=opts.LegendOpts(
                pos_top="50%"
            )
        )
    )
    
    # 把兩張表格合在一起
    grid = (
        Grid()
        .add(bar, grid_opts=opts.GridOpts(pos_bottom="60%"))
        .add(line, grid_opts=opts.GridOpts(pos_top="60%"))
    )
    
    return grid

week_chart().render_notebook()

KeyError: 'Date'